In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import re

## Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. One category of liquidated damages is related to chronic problems in any category of service at the same premises. A chronic problem is defined as more than one missed pickup for any address. The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

After determining the total amount of damages, you can look at other questions:

* What other types of complaints are there?
* Are there any geospatial analysis you can do?  Which visualizations can you create?
* How do metro crews compare to the contractor's performance?
* How much does each trash hauler owe?
* What were to total missed pickup by route?

In [23]:
#import the dataset to dataframe and check shape 
trash_hauler = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')
trash_hauler.shape

(20226, 13)

In [24]:
trash_hauler.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [83]:
Red_River = trash_hauler[trash_hauler["Trash Hauler"] == "RED RIVER"]
Red_River.shape

(14395, 13)

In [86]:
# Function to check for "Missed" or "missed"\s"Pickup" or "pickup" in the Request column
def missed_pickup(col):
    return bool(re.search(r'[Mm]issed\s[Pp]ickup', col))

# Apply the function 
missed_pickups = Red_River[Red_River['Request'].apply(missed_pickup)]
missed_pickups.shape

(10715, 13)

In [87]:
#chronic problems
adress_problem = {}
for adress in missed_pickups["Incident Address"]:
    if adress in adress_problem.keys():
        adress_problem[adress] +=1
    else:
        adress_problem[adress] = 1       
        

In [88]:
#check if all rows are returned
sum(list(adress_problem.values()))

10715

In [124]:
total_damage = 0
chronic_problem = 0
adress_affected = 0
for key, value in adress_problem.items():
    if value > 1:
        total_damage += (value-1)*200
        chronic_problem += value
        adress_affected += 1
        
print(f'Overall {adress_affected:,} adresses were affected due to chronic problem.\n This happened {chronic_problem:,} times and the total \
amount of damage due to chronic problem is $ {total_damage:,.2f}.')       


Overall 1,551 adresses were affected due to chronic problem.
 This happened 4,075 times and the total amount of damage due to chronic problem is $ 504,800.00.


#### What other types of complaints are there?

In [125]:
complaints_rr = Red_River.groupby('Request').size().reset_index(name='count')
complaints_rr = complaints_rr.sort_values("count", ascending = False)
total_complaints = sum(complaints_rr['count'])

In [126]:
complaints_rr['perecentage'] = round((complaints_rr['count']/total_complaints)*100,2)

In [127]:
complaints_rr

,Request,count,perecentage
2,Trash - Curbside/Alley Missed Pickup,10715,74.44
1,Trash - Backdoor,2196,15.26
3,Trash Collection Complaint,1287,8.94
0,Damage to Property,197,1.37


#### Are there any geospatial analysis you can do? Which visualizations can you create?

In [143]:
zipcodes = gpd.read_file('..\..\..\project_ 2\geospatial-1-1-da14-H-Getu\data/zipcodes.geojson')

In [176]:
#convert the Red_River DataFrame to GeoDataFrame
Red_River_geo = gpd.GeoDataFrame(Red_River,geometry = gpd.points_from_xy(Red_River.LONGITUDE, Red_River.LATITUDE))
Red_River_geo.geometry.centroid

0        POINT (-86.81539 36.21729)
1        POINT (-86.83810 36.21347)
2        POINT (-86.88556 36.14292)
4        POINT (-86.87499 36.15486)
5        POINT (-86.73398 36.09814)
                    ...            
20220    POINT (-86.67186 36.18948)
20221    POINT (-86.63397 36.06913)
20223    POINT (-86.67165 36.18564)
20224    POINT (-86.62003 36.05464)
20225    POINT (-86.63333 36.08368)
Length: 14395, dtype: geometry

In [179]:
center =  Red_River_geo.geometry.centroid[20220]
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

In [202]:
#create a map
map_rr = folium.Map(location =  area_center, zoom_start = 12)

    for row_index, row_values in Red_River_geo.iterrows():
        loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
        icon=folium.Icon(color="blue",icon="bus", prefix='fa')
        marker = folium.Marker( location = loc)
        marker.add_to(map_rr)
        map_rr.save('../maps/map_rr.html')

#display our map
map_rr

IndentationError: unexpected indent (2002812729.py, line 4)

In [183]:
Red_River_geo.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,geometry
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292,POINT (-86.81539 36.21729)
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470,POINT (-86.83810 36.21347)
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923,POINT (-86.88556 36.14292)
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861,POINT (-86.87499 36.15486)
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140,POINT (-86.73398 36.09814)


#### How do metro crews compare to the contractor's performance?


In [263]:
all_haulers = trash_hauler.groupby(["Trash Hauler", "Request"]).size().reset_index( name = "count")
all_haulers['Trash Hauler'] = all_haulers['Trash Hauler'].apply(lambda x: re.sub(r'Metro', 'METRO', x))

In [265]:
all_haulers = (pd.pivot_table(all_haulers, values='count', index='Trash Hauler', columns='Request', aggfunc='sum')
.sort_values(by = 'Trash - Curbside/Alley Missed Pickup', ascending=False))

In [266]:
all_haulers

Request,Damage to Property,Trash - Backdoor,Trash - Curbside/Alley Missed Pickup,Trash Collection Complaint
Trash Hauler,,,,
RED RIVER,197,2196,10715,1287
METRO,33,241,2688,618
WASTE IND,16,82,1019,233


#### How much does each trash hauler owe?